In [1]:
import pandas as pd
import numpy as np

In [46]:
train = pd.read_csv('/content/drive/MyDrive/CarPrice/train.csv')
test = pd.read_csv('/content/drive/MyDrive/CarPrice/test.csv')
sub = pd.read_csv('/content/drive/MyDrive/CarPrice/sample_submission.csv')

In [3]:
from sklearn.preprocessing import LabelEncoder

In [47]:
ordinal_features = ['브랜드', '차량모델명', '판매도시', '판매구역']

for feature in ordinal_features:
    le = LabelEncoder()
    le = le.fit(train[feature])
    train[feature] = le.transform(train[feature])

    for label in np.unique(test[feature]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    test[feature] = le.transform(test[feature])

In [34]:
train.head()

,ID,생산년도,모델출시년도,브랜드,차량모델명,판매도시,판매구역,주행거리,배기량,압축천연가스(CNG),경유,가솔린,하이브리드,액화석유가스(LPG),가격
0,TRAIN_00000,2018,2014,16,47,1214,12,85231,999,0,0,1,0,0,51.74
1,TRAIN_00001,2010,2006,17,20,2137,13,135000,1598,0,0,1,0,0,41.47
2,TRAIN_00002,2002,2002,9,36,785,15,255223,1796,0,0,1,0,0,17.81
3,TRAIN_00003,2006,2001,11,133,546,15,238000,2184,0,1,0,0,0,18.20
4,TRAIN_00004,2007,2007,3,25,1839,5,251000,1910,0,1,0,0,0,17.55


In [12]:
X = train.drop(['ID', '가격'], axis=1)
y = train['가격']

In [7]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 9.3 MB/s eta 0:00:00


In [8]:
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [11]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
xgb_model = xgb.XGBRegressor()

xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_valid)

mae = mean_absolute_error(y_valid, y_pred)
print(f"Mean Absolute Error (MAE): {mae}")

Mean Absolute Error (MAE): 6.169606240492864


In [ ]:
lgb_model = lgb.LGBMRegressor()

# Train the model
lgb_model.fit(X_train, y_train)

# Make predictions on the validation set
y_pred = lgb_model.predict(X_valid)

# Calculate the mean absolute error (MAE)
mae = mean_absolute_error(y_valid, y_pred)
print(f"Mean Absolute Error (MAE): {mae}")

Mean Absolute Error (MAE): 6.762146794426452


In [ ]:
catboost_model = CatBoostRegressor()

# Train the model
catboost_model.fit(X_train, y_train)

# Make predictions on the validation set
y_pred = catboost_model.predict(X_valid)

# Calculate the mean absolute error (MAE)
mae = mean_absolute_error(y_valid, y_pred)
print(f"Mean Absolute Error (MAE): {mae}")

In [ ]:
test = test.drop('ID', axis=1)

In [ ]:
pred = catboost_model.predict(test)

In [ ]:
sub['가격'] = pred

In [ ]:
sub.to_csv('cat.csv', index = False)

CatBoost + optuna

Best Parameters: {'boosting_type': 'Plain', 'iterations': 1674, 'learning_rate': 0.10573424441351864, 'depth': 8, 'l2_leaf_reg': 0.00026060065975250495, 'random_strength': 1.8409010826838174e-05, 'bagging_temperature': 0.01757564674119713, 'border_count': 113}

Best MAE: 5.882816248203601

In [14]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.4 MB/s eta 0:00:00


In [21]:
from catboost import Pool
import optuna
from tqdm import tqdm

In [ ]:
train_pool = Pool(X_train, y_train)
valid_pool = Pool(X_valid, y_valid)

In [ ]:
def objective(trial):
    params = {
        'loss_function': 'MAE',
        'eval_metric': 'MAE',
        'random_seed': 42,
        'task_type': 'GPU',
        'boosting_type': trial.suggest_categorical('boosting_type', ['Plain', 'Ordered']),
        'iterations': trial.suggest_int('iterations', 500, 2000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-5, 1e2),
        'random_strength': trial.suggest_loguniform('random_strength', 1e-5, 1e2),
        'bagging_temperature': trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'use_best_model': True,
        'early_stopping_rounds': 100,
        'verbose': False
    }

    # Train the CatBoost model with the current set of parameters
    model = CatBoostRegressor(**params)
    model.fit(train_pool, eval_set=valid_pool)

    # Make predictions on the validation set
    y_pred = model.predict(X_valid)

    # Calculate the mean absolute error (MAE)
    mae = mean_absolute_error(y_valid, y_pred)
    return mae

In [ ]:
pip install --upgrade tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
study = optuna.create_study(direction='minimize')
n_trials = 100
with tqdm(total=n_trials) as pbar:
    def update_pbar(_, __):
        pbar.update(1)

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials, callbacks=[update_pbar])

In [ ]:
best_params = study.best_params
best_score = study.best_value
print("Best Parameters:", best_params)
print("Best MAE:", best_score)

Best Parameters: {'boosting_type': 'Plain', 'iterations': 1674, 'learning_rate': 0.10573424441351864, 'depth': 8, 'l2_leaf_reg': 0.00026060065975250495, 'random_strength': 1.8409010826838174e-05, 'bagging_temperature': 0.01757564674119713, 'border_count': 113}
Best MAE: 5.882816248203601


In [ ]:
final_model = CatBoostRegressor(**best_params)
final_model.fit(train_pool)

In [ ]:
test_cat_optuna = test
pred = final_model.predict(test_cat_optuna)

In [ ]:
sub['가격'] = pred
sub.to_csv('cat_opnuta.csv', index = False)

Catboost+optuna+scaling
mae = 10.84


In [48]:
train.head()

,ID,생산년도,모델출시년도,브랜드,차량모델명,판매도시,판매구역,주행거리,배기량,압축천연가스(CNG),경유,가솔린,하이브리드,액화석유가스(LPG),가격
0,TRAIN_00000,2018,2014,16,47,1214,12,85231,999,0,0,1,0,0,51.74
1,TRAIN_00001,2010,2006,17,20,2137,13,135000,1598,0,0,1,0,0,41.47
2,TRAIN_00002,2002,2002,9,36,785,15,255223,1796,0,0,1,0,0,17.81
3,TRAIN_00003,2006,2001,11,133,546,15,238000,2184,0,1,0,0,0,18.20
4,TRAIN_00004,2007,2007,3,25,1839,5,251000,1910,0,1,0,0,0,17.55


In [52]:
from sklearn.preprocessing import MinMaxScaler

In [53]:
scaler  = MinMaxScaler()
columns_to_scale = ['주행거리','배기량']
train[columns_to_scale] = scaler.fit_transform(train[columns_to_scale])

In [55]:
X = train.drop(['ID', '가격'], axis=1)
y = train['가격']

In [56]:
X.head()

,생산년도,모델출시년도,브랜드,차량모델명,판매도시,판매구역,주행거리,배기량,압축천연가스(CNG),경유,가솔린,하이브리드,액화석유가스(LPG)
0,2018,2014,16,47,1214,12,0.034092,0.022857,0,0,1,0,0
1,2010,2006,17,20,2137,13,0.054000,0.133272,0,0,1,0,0
2,2002,2002,9,36,785,15,0.102089,0.169770,0,0,1,0,0
3,2006,2001,11,133,546,15,0.095200,0.241290,0,1,0,0,0
4,2007,2007,3,25,1839,5,0.100400,0.190783,0,1,0,0,0


In [57]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [58]:
train_pool = Pool(X_train, y_train)
valid_pool = Pool(X_valid, y_valid)

In [59]:
def objective(trial):
    params = {
        'loss_function': 'MAE',
        'eval_metric': 'MAE',
        'random_seed': 42,
        'task_type': 'GPU',
        'boosting_type': trial.suggest_categorical('boosting_type', ['Plain', 'Ordered']),
        'iterations': trial.suggest_int('iterations', 500, 2000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-5, 1e2),
        'random_strength': trial.suggest_loguniform('random_strength', 1e-5, 1e2),
        'bagging_temperature': trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'use_best_model': True,
        'early_stopping_rounds': 100,
        'verbose': False
    }

    # Train the CatBoost model with the current set of parameters
    model = CatBoostRegressor(**params)
    model.fit(train_pool, eval_set=valid_pool)

    # Make predictions on the validation set
    y_pred = model.predict(X_valid)

    # Calculate the mean absolute error (MAE)
    mae = mean_absolute_error(y_valid, y_pred)
    return mae

In [60]:
study = optuna.create_study(direction='minimize')
n_trials = 100
with tqdm(total=n_trials) as pbar:
    def update_pbar(_, __):
        pbar.update(1)

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials, callbacks=[update_pbar])

[I 2023-06-14 12:09:24,299] A new study created in memory with name: no-name-e4c4c522-926e-428e-bf8d-d9927d1186a7
  0%|          | 0/100 [00:00<?, ?it/s][I 2023-06-14 12:09:24,305] A new study created in memory with name: no-name-f3e6fc3f-5d33-49aa-82d9-a87edf7eff70
<ipython-input-59-a4db0c690896>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
<ipython-input-59-a4db0c690896>:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-5, 1e2),
<ipython-input-59-a4db0c690896>:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feat

In [61]:
best_params = study.best_params
best_score = study.best_value
print("Best Parameters:", best_params)
print("Best MAE:", best_score)

Best Parameters: {'boosting_type': 'Plain', 'iterations': 1504, 'learning_rate': 0.049781029418395906, 'depth': 10, 'l2_leaf_reg': 97.47680654080587, 'random_strength': 3.751312855799012e-05, 'bagging_temperature': 0.04091707530280295, 'border_count': 154}
Best MAE: 5.829141933102167


In [62]:
best_model = CatBoostRegressor(**best_params, task_type='GPU', devices='0', verbose=False)
best_model.fit(train_pool)

In [63]:
test = test.drop(['ID'], axis=1)
columns_to_scale = ['주행거리','배기량']

test[columns_to_scale] = scaler.fit_transform(test[columns_to_scale])
pred = best_model.predict(test)

In [64]:
sub['가격'] = pred

In [65]:
sub.to_csv('cat+scaling.csv', index = False)

Catboost+optuna+scaling+kfold

In [15]:
import optuna
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm

In [16]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [17]:
def objective(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 0.1, 10),
        'random_strength': trial.suggest_loguniform('random_strength', 0.1, 10),
        'border_count': trial.suggest_int('border_count', 5, 255),
        'task_type': 'GPU',
        'devices': '0',  # Specify the GPU device index
        'verbose': False
    }

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    mae_scores = []

    for train_index, val_index in kf.split(X_scaled):
        X_train, X_val = X_scaled[train_index], X_scaled[val_index]
        y_train, y_val = y[train_index], y[val_index]

        model = CatBoostRegressor(**params)
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=50, verbose=False)

        y_val_pred = model.predict(X_val)
        mae = mean_absolute_error(y_val, y_val_pred)
        mae_scores.append(mae)

    return sum(mae_scores) / len(mae_scores)

In [ ]:
study = optuna.create_study(direction='minimize')
n_trials = 100
with tqdm(total=n_trials) as pbar:
    def update_pbar(_, __):
        pbar.update(1)

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials, callbacks=[update_pbar])

In [ ]:
best_params = study.best_params
best_mae = study.best_value
print("Best Parameters:", best_params)
print("Best MAE:", best_mae)

In [ ]:
best_model = CatBoostRegressor(**best_params, task_type='GPU', devices='0', verbose=False)
best_model.fit(X_scaled, y)

In [ ]:
test_data = pd.read_csv('test.csv')
test_data_scaled = scaler.transform(test_data)
y_test_pred = best_model.predict(test_data_scaled)

xgboost+optuna

Best Parameters: {'booster': 'gbtree', 'eta': 0.09773787198485948, 'max_depth': 10, 'subsample': 0.6537116324250624, 'colsample_bytree': 0.9736768668296801, 'lambda': 2.571247074857165, 'alpha': 3.739545080324916e-05}

Best MAE: 5.997338770930938

In [ ]:
def objective(trial):
    params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'mae',
        'booster': trial.suggest_categorical('booster', ['gbtree', 'dart']),
        'eta': trial.suggest_loguniform('eta', 0.001, 0.1),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 1.0),
        'lambda': trial.suggest_loguniform('lambda', 1e-5, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-5, 10.0),
        'seed': 42,
        'verbosity': 0,
        'tree_method': 'gpu_hist'  # Set the tree_method to 'gpu_hist' for GPU acceleration
    }

    # Train and evaluate the model using the current set of parameters
    model = xgb.XGBRegressor(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_valid)
    mae = mean_absolute_error(y_valid, y_pred)

    return mae

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
study = optuna.create_study(direction='minimize')
n_trials = 100
with tqdm(total=n_trials) as pbar:
    def update_pbar(_, __):
        pbar.update(1)

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials, callbacks=[update_pbar])

In [ ]:
best_params = study.best_params
best_score = study.best_value
print("Best Parameters:", best_params)
print("Best MAE:", best_score)

Best Parameters: {'booster': 'gbtree', 'eta': 0.09773787198485948, 'max_depth': 10, 'subsample': 0.6537116324250624, 'colsample_bytree': 0.9736768668296801, 'lambda': 2.571247074857165, 'alpha': 3.739545080324916e-05}
Best MAE: 5.997338770930938


lgboost+optuna

Best Parameters: {'num_leaves': 96, 'learning_rate': 0.09833136097882149, 'feature_fraction': 0.933552804860751, 'bagging_fraction': 0.7528646718210341, 'bagging_freq': 7, 'lambda_l1': 4.485925018221742, 'lambda_l2': 0.00010634446495247863}

Best MAE: 6.12850072951182

In [ ]:
import lightgbm as lgb

In [ ]:
def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'mae',
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.6, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.6, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-5, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-5, 10.0),
        'seed': 42,
        'gpu_device_id': 0  # Specify the GPU device ID to be used
    }

    # Train and evaluate the model using the current set of parameters
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data)
    model = lgb.train(params, train_data, valid_sets=[train_data, valid_data], verbose_eval=False)
    y_pred = model.predict(X_valid)
    mae = mean_absolute_error(y_valid, y_pred)

    return mae

In [ ]:
study = optuna.create_study(direction='minimize')
n_trials = 100
with tqdm(total=n_trials) as pbar:
    def update_pbar(_, __):
        pbar.update(1)

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials, callbacks=[update_pbar])

In [ ]:
!git clone --recursive https://github.com/Microsoft/LightGBM

In [ ]:
%cd /content/LightGBM

/content/LightGBM


In [ ]:
!mkdir build

In [ ]:
!cmake -DUSE_GPU=1

In [ ]:
!make -j$(nproc)
!sudo apt-get -y install python-pip
!sudo -H pip install setuptools pandas numpy scipy scikit-learn -U

In [ ]:
%cd /content/LightGBM/python-package/
!sudo python setup.py install --precompile

In [ ]:
best_params = study.best_params
best_mae = study.best_value
print("Best Parameters:", best_params)
print("Best MAE:", best_mae)

Best Parameters: {'num_leaves': 96, 'learning_rate': 0.09833136097882149, 'feature_fraction': 0.933552804860751, 'bagging_fraction': 0.7528646718210341, 'bagging_freq': 7, 'lambda_l1': 4.485925018221742, 'lambda_l2': 0.00010634446495247863}
Best MAE: 6.12850072951182
